In [1]:
##Opening Data And saving

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls '/content/drive/Team Drives/Deep Learning compartido/Proyecto/'
myDrive = '/content/drive/Team Drives/Deep Learning compartido/Proyecto/data/'
myRoot = '/content/drive/Team Drives/Deep Learning compartido/Proyecto/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
'A Beginner'\''s Guide to Generative Adversarial Networks (GANs) | Skymind.desktop'
 backup
'Captura de 2019-06-12 12-46-38.png'
 data
'Final Project.gslides'
 hardikbansal_github_io_CycleGANBlog.pdf
'Image _translate2(3).ipynb'
'Image _translate2(4).ipynb'
'Image _translate2.ipynb'
'Image _translate.ipynb'
 TRANSFER
 ujjwalkarn_me_2016_08_11_intuitive_explanation_convnets.pdf
 Untitled0.ipynb


In [0]:
import os
import numpy as np
import torch
from torch import nn, optim

import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision.utils import save_image
from torchvision import datasets, transforms


from PIL import Image
import cv2
import matplotlib.pyplot as plt

**PROGRESS BAR**

In [0]:
def progressbar(current,limit,nbefore,description,enddescription):
  #█
  print("\b"*nbefore)
  nbar = 20
  bar = int(current*nbar/limit)
  cadena = "["+str(current)+"|"+str(limit)+"]["+"#"*(bar)+" "*(nbar-bar)+"]" + description
  ncadena = len(cadena)
  print(cadena,end="")
  if(current == limit):
    print(enddescription)
  
  return(ncadena+1)

**TRANSFER LEARNING**

In [4]:
import torchvision.models as models
resnet50_model = models.resnet50(pretrained=True) #We need pretrained resnet
for param in resnet50_model.parameters():
  param.requires_grad = False

class _Resnet50(nn.Module):
            def __init__(self):
                super( _Resnet50, self).__init__()
                self.features = nn.Sequential(
                    *list(resnet50_model.children())[:1] #Taking the firstlayer of pretrained resnet
                )
            def forward(self, x):
                x = self.features(x)
                return x

model = _Resnet50().cuda()

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 40730233.31it/s]


**Generator**

In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
               
        #ENCODER
        
        self.convE1 = nn.Conv2d(3, 64, 7, 2, 3)     #Use that if no pretrained model are required #128
        self.convE2 = nn.Conv2d(64, 128, 3, 2, 1)    #64
        self.convE3 = nn.Conv2d(128, 128, 3, 2, 1)   #32
        
        self.bnE1 = nn.BatchNorm2d(64)
        self.bnE2 = nn.BatchNorm2d(128)
        self.bnE3 = nn.BatchNorm2d(128)        
        
        #BOTTLENECK
        
        self.convT1 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT2 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT3 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT4 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT5 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT6 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT7 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT8 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT9 = nn.Conv2d(128, 128, 5, 1, 2)   #32
        self.convT10 = nn.Conv2d(128, 128, 5, 1, 2)  #32
        
        self.bnT1 = nn.BatchNorm2d(128)
        self.bnT2 = nn.BatchNorm2d(128)
        self.bnT3 = nn.BatchNorm2d(128)
        self.bnT4 = nn.BatchNorm2d(128)
        self.bnT5 = nn.BatchNorm2d(128)
        self.bnT6 = nn.BatchNorm2d(128)
        self.bnT7 = nn.BatchNorm2d(128)
        self.bnT8 = nn.BatchNorm2d(128)
        self.bnT9 = nn.BatchNorm2d(128)
        self.bnT10 = nn.BatchNorm2d(128)
        
        
        
        #DECODER
        self.convD1 = nn.ConvTranspose2d(128, 128, 4, 2, 1)    #64
        self.convD2 = nn.ConvTranspose2d(128, 64, 4, 2, 1)     #128
        self.convD3 = nn.ConvTranspose2d(64, 3, 8, 2, 3)       #256
        
        self.bnD1 = nn.BatchNorm2d(128)
        self.bnD2 = nn.BatchNorm2d(64)
        self.bnD3 = nn.BatchNorm2d(3)

        self.sig = nn.Sigmoid()


    def forward(self, x):
        
        out = model(x) #Using the first layer of resnet pretrained model for clasification
        #out = self.relu(self.bnE1(self.convE1(x)))  #Use that if no pretrained model are required
        out = self.sig(self.bnE2(self.convE2(out)))
        out = self.sig(self.bnE3(self.convE3(out)))
        
        #INTERNAL RESNET LAYER
        out_ = self.sig(self.bnT1(self.convT1(out)))
        out = out_ + out
        out_ = self.sig(self.bnT2(self.convT2(out)))
        out = out_ + out
        out_ = self.sig(self.bnT3(self.convT3(out)))        
        out = out_ + out
        out_ = self.sig(self.bnT4(self.convT4(out)))
        out = out_ + out
        out_ = self.sig(self.bnT5(self.convT5(out)))
        out = out_ + out
        out_ = self.sig(self.bnT6(self.convT6(out)))
        out = out_ + out
        
        #DECODING IMAGE
        out = self.sig(self.bnD1(self.convD1(out)))
        out = self.sig(self.bnD2(self.convD2(out)))
        out = self.sig(self.bnD3(self.convD3(out)))

        return out

**Discriminator**

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        
        #TRANSFER FEATURE EXTRACTION
        #DECISION CONVOLUTION
        
        self.sig = nn.Sigmoid()
        
        
        self.conv1 = nn.Conv2d(3, 64, 4, 2, 1)      #Use that if not pretrained one is required#128
        self.conv2 = nn.Conv2d(64, 128, 4, 2, 1)     #64
        self.conv3 = nn.Conv2d(128, 256, 4, 2, 1)    #32
        self.conv4 = nn.Conv2d(256, 512, 4, 2, 1)    #16
        self.conv5 = nn.Conv2d(512, 1, 3, 1, 1)      #16
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(512)
        self.bn5 = nn.BatchNorm2d(1)
        
        self.fc = nn.Linear(16*16,1)

        

    def forward(self, x):
        
        out = model(x) #Using the first layer of resnet pretrained model for clasification
        #out = self.relu(self.bn1(self.conv1(x)))
        out = self.sig(self.bn2(self.conv2(out)))
        out = self.sig(self.bn3(self.conv3(out)))
        out = self.sig(self.bn4(self.conv4(out)))
        out = self.sig(self.bn5(self.conv5(out)))
        
        
        out = self.fc(out.view(out.size()[0],-1))
        out = self.sig(out)
        
        return out
    
    def hook(self, module, input, output):
        self.outputs[0]  = output

**DATASET TRANSFORMATION**

In [13]:

#Taking the dataset with mini batches and aplying the transformation
BATCH = 8
trans = transforms.ToPILImage()

data_transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor()
])

datasetB = datasets.ImageFolder(root=myDrive+'DATASETS/RETRATOS/',
                                           transform=data_transform)
dataset_loaderB = torch.utils.data.DataLoader(datasetB,
                                             batch_size=BATCH, shuffle=True,
                                             num_workers=8)

datasetA = datasets.ImageFolder(root=myDrive+'DATASETS/FOTOS/',
                                           transform=data_transform)
dataset_loaderA = torch.utils.data.DataLoader(datasetA,
                                             batch_size=BATCH, shuffle=True,
                                             num_workers=8)

print(len(datasetB))

606


**TRAIN**

In [0]:
import torchvision.models as models

G_AB = Generator().cuda()
G_BA = Generator().cuda()
DA = Discriminator().cuda()
DB = Discriminator().cuda()

#--------------------Load our Trained Model-----------------------------
G_AB.load_state_dict(torch.load(myDrive+'DATASETS/MODEL/G_AB3.pth'))
G_AB.eval()
G_BA.load_state_dict(torch.load(myDrive+'DATASETS/MODEL/G_BA3.pth'))
G_BA.eval()
DA.load_state_dict(torch.load(myDrive+'DATASETS/MODEL/DA3.pth'))
DA.eval()
DB.load_state_dict(torch.load(myDrive+'DATASETS/MODEL/DB3.pth'))
DB.eval()
#--------------------/Load our Trained Model-----------------------------


optimizer_G_AB = optim.Adam(G_AB.parameters(), lr=0.0002)
optimizer_G_BA = optim.Adam(G_BA.parameters(), lr=0.0002)
optimizer_DA = optim.Adam(DA.parameters(), lr=0.0002)
optimizer_DB = optim.Adam(DB.parameters(), lr=0.0002)


criterion = nn.MSELoss()
criterion_D = nn.BCELoss()


#Realy = Variable(torch.ones(4,1)).cuda()
#Fakey = Variable(torch.zeros(4,1)).cuda()
#label = torch.cat((Realy,Fakey),0)

alfa = 10 #for cycle loss 
beta = 0.01  #for indentity loss
epochs = 500

#Creating a dictionary of losses for tracking the model 
loss = {"cycLoss":[],"G_AB_Loss":[],"G_BA_Loss":[],"DB_Loss":[],"DA_Loss":[]}

print("Training ...")


for epoch_idx in range(epochs):
  nbefore = 0  #Parameter for the progressbar status
  processed_batches = 1   #Parameter for the progressbar status
  for b,a in zip(dataset_loaderB,dataset_loaderA):
    #Dataset length of both domain are diferent so we need continue the iteration until we reach max data
    if(b[0].size()[0] != a[0].size()[0]):
      continue
    tmp = [a[0],b[0]]
    G_AB.train()
    G_BA.train()
    DB.train()
    DA.train()
    
    b = b[0].cuda()
    a = a[0].cuda()

    #First Cycle
    outG_AB_cycA = G_AB(a)
    outDB_cycA_Real = DB(b)
    outDB_cycA_Fake = DB(outG_AB_cycA)
    outG_BA_cycA = G_BA(outG_AB_cycA)
    #Second Cycle
    outG_BA_cycB = G_BA(b)
    outDA_cycB_Real = DA(a)
    outDA_cycB_Fake = DA(outG_BA_cycB)
    outG_AB_cycB = G_AB(outG_BA_cycB)
    
    #Make the identity constraint for avoiding full noise output
    same_B = G_BA(b)
    loss_identity_B = criterion(same_B, b)*beta
    same_A = G_AB(a)
    loss_identity_A = criterion(same_A, a)*beta
    
    #Build a labels for the discriminator loss function
    Realy = Variable(torch.ones(b.size()[0],1)).cuda()
    Fakey = Variable(torch.zeros(b.size()[0],1)).cuda()
    
    #Preparing loss functions
    cycLoss = criterion(outG_BA_cycA,a) + criterion(outG_AB_cycB,b)
    DB_Loss = ( criterion_D(outDB_cycA_Real,Realy) + criterion_D(outDB_cycA_Fake,Fakey) )*0.5
    DA_Loss = ( criterion_D(outDA_cycB_Real,Realy) + criterion_D(outDA_cycB_Fake,Fakey) )*0.5
    G_AB_Loss = criterion(outDB_cycA_Fake,Realy) + alfa*cycLoss
    G_BA_Loss = criterion(outDA_cycB_Fake,Realy) + alfa*cycLoss
    Gtot = loss_identity_B + loss_identity_A + criterion(outDA_cycB_Fake,Realy) + criterion(outDB_cycA_Fake,Realy) + alfa*cycLoss
    
    #Update network weights
    optimizer_G_AB.zero_grad()
    optimizer_G_BA.zero_grad()
    Gtot.backward(retain_graph=True)
    optimizer_G_BA.step()
    optimizer_G_AB.step()
    
    optimizer_DA.zero_grad()
    DA_Loss.backward()
    optimizer_DA.step()
    
    optimizer_DB.zero_grad()
    DB_Loss.backward()
    optimizer_DB.step() 
    
    #Saving losses of each iteration to plot it
    loss["cycLoss"].append(cycLoss.item())
    loss["G_AB_Loss"].append(G_AB_Loss.item()) 
    loss["G_BA_Loss"].append(G_BA_Loss.item()) 
    loss["DB_Loss"].append(DA_Loss.item()) 
    loss["DA_Loss"].append(DB_Loss.item()) 
    
    #Relevant info to show while network are training
    loss_info = "  Loss -> cycLoss:"+str(cycLoss.item())+" ,G_AB:"+str(G_AB_Loss.item())+" ,G_BA:"+str(G_BA_Loss.item())+" ,DA:"+str(DA_Loss.item())+" ,DB:"+str(DB_Loss.item())
    #Update progress bar to see status of the network training process
    nbefore = progressbar(processed_batches,int(len(datasetB)/BATCH),nbefore,"","Epoch: "+str(epoch_idx)+loss_info) 
    processed_batches += 1
 

  #After each epoch saves the parameters learned
  torch.save(G_AB.state_dict(), myDrive+'DATASETS/MODEL/G_AB3.pth')
  torch.save(G_BA.state_dict(), myDrive+'DATASETS/MODEL/G_BA3.pth')
  torch.save(DA.state_dict(), myDrive+'DATASETS/MODEL/DA3.pth')
  torch.save(DB.state_dict(), myDrive+'DATASETS/MODEL/DB3.pth')
  
  #Each epoch multiple of 10 are ploted with the first image and a plot showing the losses during that period
  if( (epoch_idx%10) == 0 ):
    #All the following code are only for data representation
    print('\n=============== EPOCH {} ================='.format(epoch_idx))

    print("CYC_LOSS: ",cycLoss)
    print("GAB_LOSS: ",G_AB_Loss)
    print("GBA_LOSS: ",G_BA_Loss)
    print("DB_LOSS: ",DB_Loss)
    print("DA_LOSS: ",DA_Loss)

    
    plt.figure()
    plt.subplot(231)
    plt.imshow(trans(tmp[0][0].detach().cpu()))
    plt.subplot(232)
    plt.imshow(trans(outG_AB_cycA[0].detach().cpu()))
    plt.subplot(233)
    plt.imshow(trans(outG_BA_cycA[0].detach().cpu()))
    plt.subplot(234)
    plt.imshow(trans(tmp[1][0].detach().cpu()))
    plt.subplot(235)
    plt.imshow(trans(outG_BA_cycB[0].detach().cpu()))
    plt.subplot(236)
    plt.imshow(trans(outG_AB_cycB[0].detach().cpu()))
    plt.figure()
    plt.title("Loss epoch "+str(epoch_idx))
    plt.plot(loss["cycLoss"],label="cycLoss")
    plt.plot(loss["G_AB_Loss"],label="G_AB_Loss")
    plt.plot(loss["G_BA_Loss"],label="G_BA_Loss")
    plt.plot(loss["DB_Loss"],label="DB_Loss")
    plt.plot(loss["DA_Loss"],label="DA_Loss")
    plt.legend()
    plt.show()
    del loss
    loss = {"cycLoss":[],"G_AB_Loss":[],"G_BA_Loss":[],"DB_Loss":[],"DA_Loss":[]}



















Output hidden; open in https://colab.research.google.com to view.

In [0]:
""